# Random Schedule - Varying N-TXs

Author: Steven M. Hernandez

The goal of this notebook is used to understand how changing the number of TXs affects the eavesdropper accuracy.

In [1]:
# 
# 0. Select experiment
# 
import ipywidgets as widgets
from datasets.experiment_group import ExperimentGroup

google_drive_directory = f"all_experiments/AntiEavesdrop/ttyUSB0"

experiment_group = ExperimentGroup(google_drive_directory, google_drive_directory.split("/")[-1])
experiment = experiment_group.experiments[0]

In [2]:
all_actions = [
    'doorway',
    'sitting desk',
    'standing desk',
    'closet',
    'empty room',
]

In [3]:
from incense import ExperimentLoader
from helpers.config import config

url = config("omniboard", "anti_eavesdrop.mongodbURI")
db_name = config("omniboard", "anti_eavesdrop.path").replace("/", "")

loader = ExperimentLoader(
    mongo_uri=url,
    db_name=db_name
)

def load_config(sacred_id):
    client = pymongo.MongoClient(url)
    sacred_db = client[db_name]
    runsDF = pd.DataFrame(list(sacred_db['runs'].find({
        'status': 'COMPLETED',
        '_id': sacred_id,
    }, {'captured_out': False})))
    
    assert(len(runsDF) == 1)
    
    return runsDF.iloc[0].config
    


In [4]:
from anti_eavesdrop.train_and_evaluate__multi_tx__random_schedule import TrainAndEvaluateMultiTXRandomSchedule
from helpers.config import config as c
from sacred.observers import MongoObserver
import ipywidgets as widgets
ta = widgets.Textarea(layout={'width': '100%'})
n = 0
optuna_labels = {}

tx_mac_addresses = {
    'A': '7C:9E:BD:4C:9C:5C',
    'B': 'C8:2B:96:8E:92:5C',
    'C': '94:B9:7E:C3:82:98',
    'D': '24:0A:C4:ED:30:50',
    'E': '94:B9:7E:C3:95:AC',
}

tx_best_config = {
    'A': 2137,
    'B': 2243,
    'C': 1909,
    'D': 2113,
    'E': 2219,
}

for evaluate_txs in [
    'ABCDE',
    'ABCD',
    'ABC',
    'AB',
#     'A',
    
    
    'ABCE',
    'ABDE',
    'ACDE',
    
    
    'ABD',
    'ABE',
    'ACD',
    'ACE',
    'ADE',
    
    
    'AC',
    'AD',
    'AE',
]:
    for pca in [64]:
        for tx, mac in list(tx_mac_addresses.items())[0:1]:
            sacred_id = tx_best_config[tx]
            config = load_config(sacred_id)
            config['mac_params'] = {
                'train_mac': mac,
                'evaluate_mac_list': [_mac for _tx, _mac in tx_mac_addresses.items() if _tx in evaluate_txs]
            }
            config['dataset_params']['interpolation'] = {
                'size_in_milliseconds': 50,
                'method': 'nearest',
            }

            label = f'anti_eavesdrop:random_schedule:train=TX-{tx}:evaluate={evaluate_txs}:{sacred_id}:pca={pca}'

            config['experiment_params']['experiment_group_label'] = label
            optuna_labels[label] = {
                'tx_train': tx,
                'tx_evaluate': evaluate_txs,
                'pca': pca,
            }
            s = TrainAndEvaluateMultiTXRandomSchedule().tsp(config, omniboard_config='anti_eavesdrop') + "\ntsp -u\n"
            ta.value += s
            n += 1

print(n)
display(ta)

15


Textarea(value='tsp docker-compose exec -T jupyter ipython -c "%run ../projects/anti_eavesdrop/train_and_evalu…

## Get Results

In [5]:
from incense import ExperimentLoader

url = c("omniboard", "anti_eavesdrop.mongodbURI")
db_name = c("omniboard", "anti_eavesdrop.path").replace("/", "")

loader = ExperimentLoader(
    mongo_uri=url,
    db_name=db_name
)

experiments = loader.find({
    'status': 'COMPLETED',
    'config.experiment_params.experiment_group_label': {'$in': list(optuna_labels.keys())},
})

output = []
for exp in experiments:
    label = exp.config['experiment_params']['experiment_group_label']
    d = {
#         'accuracy': exp.metrics['accuracy'].iloc[-1] * 100,
#         'label': label,
        'validation accuracy': exp.metrics['validation accuracy'].iloc[-1] * 100,
#             'tx': f"TX-{tx}",
        'tx_trained_on': optuna_labels[label]['tx_train'],
        'tx_evaluated_on': optuna_labels[label]['tx_evaluate'],
        'number_of_txs': len(optuna_labels[label]['tx_evaluate']),
#         '_id': exp._id,
    }
    output.append(d)

df = pd.DataFrame(output).sort_values(['number_of_txs', 'tx_trained_on', 'tx_evaluated_on'])
print(len(df))
display(df)

df.to_csv('/tmp/experiments_server/random_schedule__TX-A.csv')
dl_btn('/tmp/experiments_server/random_schedule__TX-A.csv', 'random_schedule__TX-A.csv')

15


,validation accuracy,tx_trained_on,tx_evaluated_on,number_of_txs
1,71.451610,A,AB,2
10,60.249269,A,AC,2
9,74.090910,A,AD,2
8,46.979472,A,AE,2
2,62.448680,A,ABC,3
7,69.325513,A,ABD,3
14,53.636366,A,ABE,3
13,59.325510,A,ACD,3
12,40.527859,A,ACE,3
11,48.739004,A,ADE,3


HTML(value='<html>\n    <head>\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    …

In [10]:
_df = df.groupby('number_of_txs').mean()
_df.reset_index(inplace=True)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=_df.number_of_txs,
    y=_df['validation accuracy'],
    text=[f"{a:.2f}%" for a in _df['validation accuracy']],
))
fig.update_layout(title="5-class Classifier")
fig.update_xaxes(title="Number of TX during Evaluation")
fig.update_yaxes(title="Accuracy", range=[0, 100])
fig.show('notebook_connected')

_df[['number_of_txs', 'validation accuracy']].to_csv('/tmp/experiments_server/eavesdropper__ntxs.csv')
dl_btn('/tmp/experiments_server/eavesdropper__ntxs.csv', 'eavesdropper__ntxs.csv')

HTML(value='<html>\n    <head>\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    …